In this example workflow, we'll analyze a cross section containing two 345 kV circuits to predict the impact of changing one circuit to a "delta" configuration. We'll also look at potential field reductions through phase optimization and through adding height to the circuits. Because this is a small modeling effort, we won't rely on the `emf.fields` excel templates, but will build the model using only Python.

In [44]:
#import emf.fields, which simulates electric and magnetic fields along
#lines perpendicular to parallel sets of power lines
import emf.fields as fld

First, we have to create the `emf.fields` objects that we need for the analysis.
We'll have 7 `Conductor objects` in each `CrossSection` object (3 for each circuit, plus one ground line), and use two different `CrossSections` to look at the effects of changing one circuit to the "delta" configuration. These two `CrossSections`, basically a before-after pair, will go into a top level `SectionBook` object for comparison, plotting, and exporting.

In [45]:
#create a Conductor objects for the first power line in circuit 1, which will
#consist of three 345 kV lines, all at the same height, and a single ground wire
c_1a = fld.Conductor('1a') #create the conductor with the name '1a'
c_1a.x = -25 #x coordinate in feet
c_1a.y = 22 #y coordinate in feet
c_1a.subconds = 1 #number of subconductors in the "bundle" (usually 1)
c_1a.d_cond = 1.5 #diameter of conductor in inches
c_1a.d_bund = 1.5 #diameter of conductor "bundle" in inches
c_1a.V = 345 #loading in volts
c_1a.I = 600 #current in amps
c_1a.phase = 240 #phase angle in degrees

In [46]:
#create the other two conductors in circuit 1 by copying and editing Conductor 1a
c_1b = c_1a.copy()
c_1b.tag = '1b'
c_1b.x = -20
c_1b.phase = 120

c_1c = c_1b.copy()
c_1c.tag = '1c'
c_1c.x = -15
c_1c.phase = 0

In [47]:
#now create the second circuit, which will be the same as the first,
#but with x coordinates reflected about x = 0
c_2a = c_1a.copy()
c_2a.tag = '2a'
c_2a.x = abs(c_2a.x)

c_2b = c_1b.copy()
c_2b.tag = '2b'
c_2b.x = abs(c_2b.x)

c_2c = c_1c.copy()
c_2c.tag = '2c'
c_2c.x = abs(c_2c.x)

Now that we've created `Conductor` objects for each of the lines in our before (all lines at the same height) and after (circuit 2 switched to a triangular configuration) models, we can put the `Conductor`s into `CrossSection` objects, which will perform the desired EMF calculations. `Conductor`s can be added to a `CrossSection` object upon its initialization.

In [48]:
#now the first CrossSection object can be created, containing the Conductor
#objects created above
before = fld.CrossSection('before', [c_1a, c_1b, c_1c, c_2a, c_2b, c_2c])
#set some identification variables
before.tag = 'Delta Test'
before.title = 'Before Delta Implementation'
#and set the sampling variables, which determine where emf values are calculated
before.max_dist = 50 #sample to 50 ft from the center of the xs
before.step = 0.5 #sample every half foot along the xs
before.lROW = -40 #left edge of power line right of way (ROW)
before.rROW = 40 #right ROW edge

In [49]:
#check that the CrossSection is complete, meaning all the necessary variables in
#its Conductor objects are set
print before.complete

(True, None, None)


In [50]:
#all set, now we'll copy and edit the "before" CrossSection to create the
#"after" version with the delta configuration
after = before.copy()
after.sheet = 'after'
after.tag = 'Delta Test'
after.title = 'After Delta Implementation'
#Conductors in a CrossSection can be accessed by their tag strings
after['2b'].y += 5
after['2c'].x = after['2a'].x
after['2c'].y = after['2a'].y + 10.0

Lastly, before looking at the numbers, we put the `CrossSection` objects into a top level `SectionBook` object, which is essential for comparing the `CrossSections`. `CrossSection`s can be added to a `SectionBook` object upon its initialization.

In [51]:
#finally, we can put the before and after CrossSections into a top level
#SectionBook object
sb = fld.SectionBook('delta-test', [before, after])

But wait! We forgot the ground wire in circuit 1! Adding a `Conductor`s to `CrossSection` is easy with `add_conductor()`.

In [52]:
#create the grounded line
ground = fld.Conductor('1g')
ground.x = -17.5
ground.y = 27.2
ground.subconds = 1
ground.d_cond = 1
ground.d_bund = 1
ground.V = 0 #grounded
ground.I = 0
ground.phase = 0

#add the ground line to the before and after xcs, which can be accessed
#by keying the SectionBook like a dict, using the CrossSection's
#sheet strings
sb['before'].add_conductor(ground)
sb['after'].add_conductor(ground)

On to the results... EMF calculations are performed by a `CrossSection` only when its `field` attribute is accessed. If a `Conductor` in the `CrossSection` is modified, that `CrossSection`'s fields are updated the next time its `field` attribute is accessed. We don't have to do anything at all to get the most up-to-date fields for a `CrossSection`. For example, if we access fields from the `before` `CrossSection`, they will automatically calculate.

In [53]:
#the fields attribute leads to a Pandas DataFrame
print sb['before'].fields.head(5)

            Bmax      Bprod         Bx        By      Emax     Eprod  \
-50.0  22.383021  22.394635  21.682692  5.601831  1.688669  1.688741   
-49.5  23.028616  23.041231  22.397989  5.406329  1.732469  1.732543   
-49.0  23.696805  23.710513  23.136263  5.186693  1.777254  1.777329   
-48.5  24.388393  24.403296  23.897784  4.941332  1.823005  1.823083   
-48.0  25.104200  25.120410  24.682748  4.668722  1.869702  1.869782   

             Ex        Ey  
-50.0  0.257093  1.669056  
-49.5  0.262819  1.712493  
-49.0  0.268446  1.756939  
-48.5  0.273940  1.802384  
-48.0  0.279262  1.848810  


Now if we change information in one of the `Conductor` objects in the `before` `CrossSection` and access the `fields` attribute again, it will automatically recalculate.

In [54]:
#Conductors can also be accessed by the order in which they were
#added to the CrossSection
sb['before'].i[0].V += 1000 #boost the Voltage!
sb['before']['1b'].I += 1000 #boost the Current!
print sb['before'].fields.head(5)

             Bmax       Bprod          Bx          By       Emax      Eprod  \
-50.0  184.338265  184.975014   98.865314  156.337474  10.019097  10.019113   
-49.5  186.573209  187.251314  101.277400  157.499025  10.265217  10.265233   
-49.0  188.852940  189.575325  103.766759  158.654541  10.517507  10.517523   
-48.5  191.178380  191.948181  106.336018  159.802238  10.775992  10.776008   
-48.0  193.550447  194.371028  108.987862  160.940183  11.040681  11.040697   

             Ex         Ey  
-50.0  1.445012   9.914361  
-49.5  1.480946  10.157845  
-49.0  1.517023  10.407542  
-48.5  1.553134  10.663495  
-48.0  1.589153  10.925730  


The fields were automatically recalculated when the `fields` property was accessed, becasue we changed some of the `Conductor` variables. We have some seriously high fields now... Let's change them back and make sure the fields changed again.

In [55]:
sb['before'].i[0].V -= 1000
sb['before']['1b'].I -= 1000
print sb['before'].fields.head(1)

            Bmax      Bprod         Bx        By      Emax     Eprod  \
-50.0  22.383021  22.394635  21.682692  5.601831  1.688669  1.688741   

             Ex        Ey  
-50.0  0.257093  1.669056  


All set. Now on to the real analysis. Let's take a look at how the magnetic fields compare between our before and after `CrossSections` by plotting them as a group. Groups of `CrossSection`s within `SectionBook`s all have the same `tag` attribute. That's why we set those `tag` attributes to the same thing when the `CrossSection`s were created (the tag is `'Delta Test'`).

In [56]:
figs = fld.plot_groups(sb, return_figs=True)
fld.close(figs['E']['Delta Test']) #get rid of the electric field plot
fld.show()

It's clear from the plot that switching to a "delta" reduces fields throughout the ROW. we can inspect the actual values of electric and magnetic fields at the ROW edges, at a glance, with the SectionBook's ROW_edge_max property.

In [57]:
print sb.ROW_edge_max

       Bmaxl      Bmaxr     Emaxl     Emaxr   sheet  \
0  40.542720  40.542720  2.692491  2.574660  before   
1  39.829126  35.332244  2.647290  2.169042   after   

                         title  
0  Before Delta Implementation  
1   After Delta Implementation  


Fields are significantly reduced on the right side of the ROW, and slightly reduced on the left side because it's farther from the second circuit. If we want to reduce fields even further, we can find the optimal phasing of the "after" CrossSection and see if it helps.

In [58]:
res, opt = fld.optimize_phasing(sb['after'], 'all')

#look at whether a phasing configuration yields a significant reduction
#in magnetic fields
fld.plot_groups_at_ROW(opt, return_figs=True)
fld.close(figs['E']['Delta Test'])
fld.show()

Looks like phase optimization of the "after" CrossSection wouldn't reduce the fields because the circuits are already in one of potentially many optimal arrangements. Let's just check the numbers.

In [59]:
print opt.ROW_edge_max

       Bmaxl      Bmaxr     Emaxl     Emaxr                     sheet  \
0  39.829126  35.332244  2.647290  2.169042          after (Original)   
1  39.380491  39.247289  2.654888  2.107660   Optimized for Bmax left   
2  39.829126  35.332244  2.647290  2.169042  Optimized for Bmax right   
3  39.829126  35.332244  2.647290  2.169042   Optimized for Emax left   
4  39.380491  39.247289  2.654888  2.107660  Optimized for Emax right   

                                   title  
0             After Delta Implementation  
1   Optimal Phasing - Bmax Left ROW Edge  
2  Optimal Phasing - Bmax Right ROW Edge  
3   Optimal Phasing - Emax Left ROW Edge  
4  Optimal Phasing - Emax Right ROW Edge  


As a last resort, we're asked how much higher we would have to raise the second circuit, the "delta", to get magnetic fields at the right ROW edge down to 30 mG. We can use `fld.target_fields()`.

In [60]:
h, adj = fld.target_fields(sb['after'], ['2a', '2b', '2c'], 0, 30, 0, 0)
print h
figs = fld.plot_groups(adj, return_figs=True)
fld.close(figs['E']['Height Adjusted'])
fld.show()

(None, 2.209329977631569, None, None)


We would only have to raise circuit 2 an extra 2.2 feet to get predicted magnetic field values at the right ROW edge down to 30 mG. That's good info. We can export the original modeling results and the height adjusted results to excel files, and be done with it.

In [61]:
sb.export()
h, adj = fld.target_fields(sb['after'], ['2a', '2b', '2c'],
                            0, 30, 0, 0, save=True)

Full SectionBook results written to: delta-test-all_results.xlsx
Maximum fields at ROW edges written to: delta-test-ROW_edge_results.csv
Optimal phasing results written to: after_height_adjustments.xlsx
